In [ ]:
###################################################################################################
# None, []
###################################################################################################
>>> a = None # reduce the ref count of the list object by 1
>>> a = [] # reduce the ref count of the list object by 1



###################################################################################################
# Containers, Iterators, Generators
###################################################################################################
# Lists https://docs.python.org/3.3/library/stdtypes.html#lists
###################################################################################################
#class list([iterable])
#Lists may be constructed in several ways:

#Using a pair of square brackets to denote the empty list: []
#Using square brackets, separating items with commas: [a, b, c] - usual way
#Using a list comprehension: [x for x in iterable] 
#Using the type constructor: list() or list(iterable)

#list('abc') returns ['a', 'b', 'c'] and 
#list( (1, 2, 3) ) returns [1, 2, 3]. 
#If no argument is given, the constructor creates a new empty list, [].
>>> l = [3, 4, 5]
>>> l2 = [3, 4, 5]
>>> l == l2 # Unlike java identity comparison, in Python it's an equality comparison, ref:https://docs.python.org/3.4/reference/expressions.html#comparisons
#True
>>> id(l)
#62086024
>>> id(l2)
#62084104

###################################################################################################
# Tuple https://docs.python.org/3.3/library/stdtypes.html#tuples
###################################################################################################
# class tuple([iterable])
# Tuples may be constructed in a number of ways:

# Using a pair of parentheses to denote the empty tuple: ()
# Using a trailing comma for a singleton tuple: a, or (a,)
# Separating items with commas: a, b, c or (a, b, c)
# Using the tuple() built-in: tuple() or tuple(iterable)


###################################################################################################
# dict https://docs.python.org/3.3/library/stdtypes.html#dict
###################################################################################################
>>> a = dict(one=1, two=2, three=3)
>>> b = {'one': 1, 'two': 2, 'three': 3}
>>> c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
>>> d = dict([('two', 2), ('one', 1), ('three', 3)])
>>> e = dict({'three': 3, 'one': 1, 'two': 2})
>>> a == b == c == d == e
True


###################################################################################################
# Iterators
###################################################################################################
# iter(object[, sentinel]) https://docs.python.org/3.3/library/functions.html#iter
# Return an iterator object. The first argument is interpreted very differently depending on the presence of the second argument. 
# Without a second argument, object must be a collection object which supports the iteration protocol (the __iter__() method), or it must 
# support the sequence protocol (the __getitem__() method with integer arguments starting at 0). If it does not support either of those protocols, 
# TypeError is raised. If the second argument, sentinel, is given, then object must be a callable object (with __call__()). The iterator created in this case will 
# call object with no arguments for each call to its __next__() method; if the value returned is equal to sentinel, StopIteration will be 
# raised, otherwise the value will be returned.

# next(iterator[, default]) https://docs.python.org/3.3/library/functions.html#next
# Retrieve the next item from the iterator by calling its __next__() method. 
# If default is given, it is returned if the iterator is exhausted, otherwise StopIteration is raised.

# **IMPORTANT https://docs.python.org/3.3/library/stdtypes.html#iterator-types

def zip(*iterables):
    # zip('ABCD', 'xy') --> Ax By
    sentinel = object()
    iterators = [iter(it) for it in iterables]
    while iterators:
        result = []
        for it in iterators:
            elem = next(it, sentinel)
            if elem is sentinel:
                return
            result.append(elem)
        yield tuple(result)

# Iterator for callable object (__call__) http://stackoverflow.com/questions/9815772/how-to-pass-arguments-to-function-of-callable-iterator-in-python
call_iter = iter(lambda:lambda x: x + 1, 100)
>>> next(call_iter)(1)
2

>>> ll = lambda:lambda x: x + 1
>>> ll()(1) # or ll.__call__()(1) or ll.__call__().__call__(1)

###################################################################################################
# Generators (generator | generator expression | iterator protocol | getitem method)
###################################################################################################
# Generator Types https://docs.python.org/3.3/library/stdtypes.html#generator-types
# Python’s generators provide a convenient way to implement the iterator protocol. If a container object’s __iter__() method 
# is implemented as a generator, it will automatically return an iterator object (technically, a generator object) supplying 
# the __iter__() and __next__() methods. More information about generators can be found in the documentation for the yield expression.

# Ref: https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/
# Generators are used to generate a series of values
#       - yield is like the return of generator functions
#       - The only other thing yield does is save the "state" of a generator function
#       - A generator is just a special type of iterator
#       - Like iterators, we can get the next value from a generator using next()
#       - for gets values by calling next() implicitly

# Ref: https://docs.python.org/2/reference/datamodel.html#special-method-names
# 3.4. Special method names
# A class can implement certain operations that are invoked by special syntax (such as arithmetic operations 
# or subscripting and slicing) by defining methods with special names. This is Python’s approach to operator 
# overloading, allowing classes to define their own behavior with respect to language operators. For instance, 
# if a class defines a method named __getitem__(), and x is an instance of this class, then x[i] is roughly 
# equivalent to x.__getitem__(i) for old-style classes and type(x).__getitem__(x, i) for new-style classes. 
# Except where mentioned, attempts to execute an operation raise an exception when no appropriate method is 
# defined (typically AttributeError or TypeError).

# generator
def uc_gen(text):
    for char in text:
        yield char.upper() # https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/
                           # End of the generator function will cause StopIteration to be automatically raised. 

# generator expression # https://www.python.org/dev/peps/pep-0289/
def uc_genexp(text):
    return (char.upper() for char in text)

# iterator protocol
class uc_iter():
    def __init__(self, text):
        self.text = text
        self.index = 0
    def __iter__(self):
        return self
    def __next__(self):
        try:
            result = self.text[self.index].upper()
        except IndexError:
            raise StopIteration # This indicates the end of the sequence
        self.index += 1
        return result # IMPORTANT: Should not have yield statement in the body of __next__()
                      # Extract the yield to another method pointed by self.generator and invoke it using next(self.generator) 

# sequence protocol
class uc_getitem():
    def __init__(self, text):
        self.text = text
    def __getitem__(self, index):
        result = self.text[index].upper()
        return result

    # Ref: https://docs.python.org/2/reference/datamodel.html#emulating-container-types
    # object.__getitem__(self, key)
    # Called to implement evaluation of self[key]. For sequence types, the accepted keys should be integers and slice objects. 
    # Note that the special interpretation of negative indexes (if the class wishes to emulate a sequence type) is up to the 
    # __getitem__() method. If key is of an inappropriate type, TypeError may be raised; if of a value outside the set of indexes 
    # for the sequence (after any special interpretation of negative values), IndexError should be raised. For mapping types, 
    # if key is missing (not in the container), KeyError should be raised.
    #
    # *** Note: for loops expect that an IndexError will be raised for illegal indexes to allow proper detection of the end of 
    #           the sequence.

# To see all four methods in action:
for iterator in [uc_gen, uc_genexp, uc_iter, uc_getitem]:
    for ch in iterator('abcde'):
        print(ch)
    print('')

# Which results in:

# A B C D E
# A B C D E
# A B C D E
# A B C D E

>>> y = uc_gen('abcdefgh')
>>> y
<generator object uc_gen at 0x0000000003B37F78>
>>> next(y)
'A'

#
# generator expression # https://www.python.org/dev/peps/pep-0289/
#
g = (x**2 for x in range(10))
print (next(g))

# is equivalent to:

def __gen(exp):
    for x in exp:
        yield x**2
g = __gen(iter(range(10)))
print (next(g))


for square in g:
    print(square)


###################################################################################################
# Generators .send()
###################################################################################################
# Ref: https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/
# In PEP 342, support was added for passing values into generators. PEP 342 gave generators the power 
# to yield a value (as before), receive a value, or both yield a value and receive a (possibly different) 
# value in a single statement.

# Statement of the form other = yield foo means, "yield foo and, when a value is sent to me, set other to that value." 
# You can "send" values to a generator using the generator's send method.
def get_primes(number):
    while True:
        if is_prime(number):
            number = yield number
        number += 1

# In this way, we can set number to a different value each time the generator yields. We can now fill 
# in the missing code in print_successive_primes:

def print_successive_primes(iterations, base=10):
    prime_generator = get_primes(base)
    prime_generator.send(None)
    for power in range(iterations):
        print(prime_generator.send(base ** power))

# Two things to note here: First, we're printing the result of generator.send, which is possible because send 
# both sends a value to the generator and returns the value yielded by the generator (mirroring how yield works 
# from within the generator function).

# Second, notice the prime_generator.send(None) line. When you're using send to "start" a generator 
# (that is, execute the code from the first line of the generator function up to the first yield statement),
# you must send None. This makes sense, since by definition the generator hasn't gotten to the first yield 
# statement yet, so if we sent a real value there would be nothing to "receive" it. Once the generator is started, 
# we can send values as we do above.
